# Resumo
Este notebook será apenas para gerar e guardar um simples modelo de classificação para ser chamado via API para consumir o mesmo

Basicamente um modelo classificador para predizer se um pokemon é ou não lendário de acordo com seu status

In [87]:
import pandas as pd
import pickle
from flask import request
import requests

In [88]:
df = pd.read_csv("Poke.csv")
df.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [89]:
df.drop(['Name','Type 1','Type 2'], axis=1, inplace=True)
df.head()

,#,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,318,45,49,49,65,65,45,1,False
1,2,405,60,62,63,80,80,60,1,False
2,3,525,80,82,83,100,100,80,1,False
3,3,625,80,100,123,122,120,80,1,False
4,4,309,39,52,43,60,50,65,1,False


## Preparando os dados

In [90]:
df['Legendary'].replace([False,True],[0,1], inplace=True)

## Gerando Modelos

In [91]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [92]:
X = df.drop('Legendary', axis=1)
y = df['Legendary']

In [93]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, random_state=123)

In [94]:
ada = AdaBoostClassifier()
grad = GradientBoostingClassifier()
rforest = RandomForestClassifier()

## Treino

In [95]:
ada.fit(Xtrain, ytrain)
grad.fit(Xtrain, ytrain)
rforest.fit(Xtrain, ytrain)

RandomForestClassifier()

In [96]:
predicaoAda = ada.predict(Xtest)
predicaoGrad = grad.predict(Xtest)
predicaoForest = rforest.predict(Xtest)

In [97]:
scoreAda = ada.score(Xtest, ytest)
scoreGrad = grad.score(Xtest, ytest)
scoreForest = rforest.score(Xtest, ytest)

## Performance

In [98]:
print(f'Random Forest: {scoreForest* 100:.0f}%\nGradient Boosting: {scoreGrad*100:.0f}%\nAdaboost: {scoreAda*100:.0f}%')

Random Forest: 97%
Gradient Boosting: 96%
Adaboost: 97%


In [99]:
print(classification_report(ytest, predicaoForest))

              precision    recall  f1-score   support

           0       0.98      0.99      0.98       184
           1       0.86      0.75      0.80        16

    accuracy                           0.97       200
   macro avg       0.92      0.87      0.89       200
weighted avg       0.97      0.97      0.97       200



## Salvando modelo escolhido na maquina local

In [100]:
local = !pwd

In [101]:
pickle.dump(rforest, open(local[0]+'/pokeModel.pkl', 'wb'))

___
## Chamando a API para fazer predições
Passando um indice do dataset para a variavel unidade sem a coluna alvo que será predita e convertemos para json para enviar para API

In [102]:
unidade = df.drop('Legendary', axis=1).sample()
unidade

,#,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation
674,613,305,55,70,40,60,40,40,5


In [103]:
df_json = unidade.to_json(orient='records')
url = 'http://0.0.0.0:5000/predict'
data = df_json
header = {'Content-type':'application/json'}
    
r = requests.post(url=url, data=data, headers=header)
r.status_code

200

## Devolvendo a predição no formato de dataframe

In [104]:
pd.DataFrame(r.json(), columns=r.json()[0].keys())

,#,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,prediction
0,613,305,55,70,40,60,40,40,5,0
